# Process WFABBA, Merge Files

<b>Summary:</b><br>
Reads in raw WFABBA GOES-16 & WFABBA GOES-17 data from individual text files and outputs as combined csv files<br>

- Parse WFABBA data from individual text files
- Merge WFABBA data into unified csv files (based off of time and satellite name)

<b>Output:</b><br>
../..<br>
└── data<br>
&emsp;&emsp;&emsp;└── processed<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── GOES-16-2019.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── GOES-17-2019.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── GOES-16-2020.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── GOES-17-2020.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── GOES-16-2021.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── GOES-17-2021.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── GOES-16-Jan-2021.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── GOES-17-Jan-2021.csv<br>

<b>Instructions:</b><br>
- See README.md in directory

<b>Areas for Improvement:</b><br>
Current processing time is about 3160 seconds. Need to consider faster processing methods.

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import pathlib
import os
import time

In [2]:
metadataColNames = ["Algorithm", "Version", "Timestamp", "Satellite", "Instrument", "FlightModel", "ScanMode", "ProductType", "FileName", "DataSource",
               "DataCreationTimestamp", "NavProjSubPtLong", "ActualSatSubPtLong", "NumFire", "MissingValueCode"]

dataColNames = ['Latitude', 'Longitude', 'Code', 'FRP', 'Fire Size', 'Fire Temp', 'Line', 'Element', 'Pixel Size',  'Obs BT4', 'Obs BT11', 'Bkg BT4', 'Bkg BT11', 'SolZen',  'SatZen', 'RelAzi', 'Eco']

allColNames = metadataColNames + dataColNames
rawDir = "../../data/raw/wfabba/"
processedDir = "../../data/processed/wfabba/"

In [3]:
starttime = time.time()
wfabba_dates = ["GOES-16-2019", "GOES-17-2019", "GOES-16-2020", "GOES-17-2020", "GOES-16-2021", "GOES-17-2021", "GOES-16-Jan-2021","GOES-17-Jan-2021"]

for wfabba_date in wfabba_dates:
    masterArr = []
    print(wfabba_date)

    targetDir = "../../data/raw/wfabba/" + wfabba_date

    # recursively visit every directory of raw data
    for root, subdirs, files in sorted(os.walk(targetDir)):
        print(root)
        print(len(masterArr))

        #if there are any files in the directory to parse
        if len(files) > 0:
            # process files with .GOES-16, .GOES-17, .GOES-16.txt, or .GOES-17.txt
            if files[0][-8:] == ".GOES-16" or files[0][-8:] == ".GOES-17" or files[0][-12:] == ".GOES-16.txt" or files[0][-12:] == ".GOES-17.txt" :
                #loop through each file
                for file in files:
                    filepath = root+"/"+file
                    
                    fo = open(filepath ,'r')
                    metadataDict = {}
                    dataArr = []
                    lineCount = 0
                    
                    try:
                        # read every line in the file
                        for line in fo:
                            
                            # read in metadata
                            if line[0:4] == "### ":
                                # logic for first 11 lines
                                if lineCount < 11:
                                    parseString = line[4:]
                                    parseString = parseString.strip()
                                    
                                    # timestamp parsing logic
                                    if parseString[0:4] == 'Date':
                                        if parseString == "Date: , Time:  UTC":
                                            break
                                        year = parseString[6:10]
                                        day_num = parseString[10:13]
                                        hr = parseString[21:23]
                                        min = parseString[24:26]
                                        sec = parseString[27:29]
                                        
                                        #break out of loop if the timestamp data is all 0's
                                        if year == "0000" and day_num == "000" and hr == "00" and min == "00" and sec == "00":
                                            break
                                        
                                        # create timestamp from time data
                                        res = dt.datetime.strptime(year + "-" + day_num + " " + hr + ":" + min + ":" + sec, "%Y-%j %H:%M:%S")
                                        metadataDict["Timestamp"] = res
                                    else:
                                        # parse column headers for data
                                        splitData = parseString.split(", ")
                                        for item in splitData:
                                            if item[-9:] == 'Algorithm':
                                                metadataDict["Algorithm"] = item
                                            else:
                                                sections = item.split(":")

                                                if sections[0] == 'Flight Model':
                                                    keyname = 'FlightModel'
                                                elif sections[0] == 'Scan Mode':
                                                    keyname = 'ScanMode'
                                                elif sections[0] == 'Product type':
                                                    keyname = 'ProductType'
                                                elif sections[0] == 'Product/L2 filename':
                                                    keyname = 'FileName'
                                                elif sections[0] == 'Data source':
                                                    keyname = 'DataSource'
                                                elif sections[0] == 'Data creation time stamp':
                                                    keyname = 'DataCreationTimestamp'
                                                elif sections[0] == 'Navigation projection subpoint longitude':
                                                    keyname = 'NavProjSubPtLong'
                                                elif sections[0] == 'Actual satellite subpoint longitude':
                                                    keyname = 'ActualSatSubPtLong'
                                                elif sections[0] == 'Number of detected fires':
                                                    keyname = 'NumFire'
                                                elif sections[0] == 'Missing value code':
                                                    keyname = 'MissingValueCode'
                                                else:
                                                    keyname = sections[0]

                                                if len(sections) > 1:
                                                    if sections[1] == '':
                                                        value = None
                                                    else:
                                                        value = sections[1].strip()
                                                    metadataDict[keyname] = value
                                    lineCount+=1
                            # parse the actual data
                            elif line[0:3] != '###':
                                line_list = line.replace(' ', '').replace("\n","").split(',')
                                app_dict = {dataColNames[i]: line_list[i] for i in range(len(dataColNames))}

                                #keep 32 <= latitudes <= 35 and -123 <= longitude <= -113
                                if (float(app_dict["Latitude"]) >= 32) and (float(app_dict["Latitude"]) <= 35) and (float(app_dict["Longitude"]) >= -123) and (float(app_dict["Longitude"]) <= -113):
                                    dataArr.append(app_dict)
                                    
                    # error handling: print out the name of the file that is causing issues and break out of loop
                    except:
                        print("Error at " + filepath)
                        break
                        
                    # combine metadata and data into master array
                    for dataDict in dataArr:
                        allDataDict = {}
                        allDataDict.update(metadataDict)
                        allDataDict.update(dataDict)
                        masterArr.append(allDataDict)
                        
                    fo.close()
    
    #create dataframe out of master array
    df = pd.DataFrame(masterArr, columns=allColNames)
    endtime = time.time()
    print(endtime-starttime)
    df.to_csv(processedDir + wfabba_date + ".csv")
    print("==========================")

GOES-16-2019
../../data/raw/wfabba/GOES-16-2019
0
../../data/raw/wfabba/GOES-16-2019/2019_06_01_152
0
../../data/raw/wfabba/GOES-16-2019/2019_06_02_153
260
../../data/raw/wfabba/GOES-16-2019/2019_06_03_154
376
../../data/raw/wfabba/GOES-16-2019/2019_06_04_155
620
../../data/raw/wfabba/GOES-16-2019/2019_06_05_156
936
../../data/raw/wfabba/GOES-16-2019/2019_06_06_157
1120
../../data/raw/wfabba/GOES-16-2019/2019_06_07_158
1235
../../data/raw/wfabba/GOES-16-2019/2019_06_08_159
1297
../../data/raw/wfabba/GOES-16-2019/2019_06_09_160
1360
../../data/raw/wfabba/GOES-16-2019/2019_06_10_161
1450
../../data/raw/wfabba/GOES-16-2019/2019_06_11_162
1537
../../data/raw/wfabba/GOES-16-2019/2019_06_12_163
1605
../../data/raw/wfabba/GOES-16-2019/2019_06_13_164
1683
../../data/raw/wfabba/GOES-16-2019/2019_06_14_165
1882
../../data/raw/wfabba/GOES-16-2019/2019_06_15_166
2157
../../data/raw/wfabba/GOES-16-2019/2019_06_16_167
2313
../../data/raw/wfabba/GOES-16-2019/2019_06_17_168
2481
../../data/raw/wfabba/

../../data/raw/wfabba/GOES-16-2019/2019_10_26_299
67255
../../data/raw/wfabba/GOES-16-2019/2019_10_27_300
68072
../../data/raw/wfabba/GOES-16-2019/2019_10_28_301
68224
../../data/raw/wfabba/GOES-16-2019/2019_10_29_302
68665
../../data/raw/wfabba/GOES-16-2019/2019_10_30_303
68680
../../data/raw/wfabba/GOES-16-2019/2019_10_31_304
69056
../../data/raw/wfabba/GOES-16-2019/2019_11_01_305
69503
../../data/raw/wfabba/GOES-16-2019/2019_11_02_306
70519
../../data/raw/wfabba/GOES-16-2019/2019_11_03_307
70603
../../data/raw/wfabba/GOES-16-2019/2019_11_04_308
70613
../../data/raw/wfabba/GOES-16-2019/2019_11_05_309
70617
../../data/raw/wfabba/GOES-16-2019/2019_11_06_310
70633
../../data/raw/wfabba/GOES-16-2019/2019_11_07_311
70683
../../data/raw/wfabba/GOES-16-2019/2019_11_08_312
70685
../../data/raw/wfabba/GOES-16-2019/2019_11_09_313
70692
../../data/raw/wfabba/GOES-16-2019/2019_11_10_314
70706
../../data/raw/wfabba/GOES-16-2019/2019_11_11_315
70725
../../data/raw/wfabba/GOES-16-2019/2019_11_12_31

../../data/raw/wfabba/GOES-17-2019/2019_08_19_231
311970
../../data/raw/wfabba/GOES-17-2019/2019_08_20_232
312193
../../data/raw/wfabba/GOES-17-2019/2019_08_21_233
312817
../../data/raw/wfabba/GOES-17-2019/2019_08_22_234
313279
../../data/raw/wfabba/GOES-17-2019/2019_08_23_235
313564
../../data/raw/wfabba/GOES-17-2019/2019_08_24_236
313602
../../data/raw/wfabba/GOES-17-2019/2019_08_25_237
313634
../../data/raw/wfabba/GOES-17-2019/2019_08_26_238
314408
../../data/raw/wfabba/GOES-17-2019/2019_08_27_239
314778
../../data/raw/wfabba/GOES-17-2019/2019_08_28_240
315363
../../data/raw/wfabba/GOES-17-2019/2019_08_29_241
315472
../../data/raw/wfabba/GOES-17-2019/2019_08_30_242
317498
../../data/raw/wfabba/GOES-17-2019/2019_08_31_243
318404
../../data/raw/wfabba/GOES-17-2019/2019_09_01_244
319895
../../data/raw/wfabba/GOES-17-2019/2019_09_02_245
320511
../../data/raw/wfabba/GOES-17-2019/2019_09_03_246
321068
../../data/raw/wfabba/GOES-17-2019/2019_09_04_247
321179
../../data/raw/wfabba/GOES-17-2

../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_09_009
165
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_10_010
174
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_11_011
226
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_12_012
236
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_13_013
248
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_14_014
252
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_15_015
252
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_16_016
257
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_17_017
277
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_18_018
301
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_19_019
311
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_20_020
314
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_21_021
314
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_22_022
336
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_23_023
359
../../data/raw/wfabba/GOES-16-2020/Part1/2020_01_24_024
378
../../data/raw/wfabba/GOES-16-2020/Part1

../../data/raw/wfabba/GOES-16-2020/Part2/2020_05_24_145
10734
../../data/raw/wfabba/GOES-16-2020/Part2/2020_05_25_146
10826
../../data/raw/wfabba/GOES-16-2020/Part2/2020_05_26_147
11038
../../data/raw/wfabba/GOES-16-2020/Part2/2020_05_27_148
11162
../../data/raw/wfabba/GOES-16-2020/Part2/2020_05_28_149
11227
../../data/raw/wfabba/GOES-16-2020/Part2/2020_05_29_150
11370
../../data/raw/wfabba/GOES-16-2020/Part2/2020_05_30_151
11643
../../data/raw/wfabba/GOES-16-2020/Part2/2020_05_31_152
12054
../../data/raw/wfabba/GOES-16-2020/Part2/2020_06_01_153
12256
../../data/raw/wfabba/GOES-16-2020/Part2/2020_06_02_154
12342
../../data/raw/wfabba/GOES-16-2020/Part2/2020_06_03_155
13150
../../data/raw/wfabba/GOES-16-2020/Part2/2020_06_04_156
13693
../../data/raw/wfabba/GOES-16-2020/Part2/2020_06_05_157
13971
../../data/raw/wfabba/GOES-16-2020/Part2/2020_06_06_158
14073
../../data/raw/wfabba/GOES-16-2020/Part2/2020_06_07_159
14506
../../data/raw/wfabba/GOES-16-2020/Part2/2020_06_08_160
14624
../../da

...

../../data/raw/wfabba/GOES-16-2021/2021_08_24_236
79217
../../data/raw/wfabba/GOES-16-2021/2021_08_25_237
79444
../../data/raw/wfabba/GOES-16-2021/2021_08_26_238
79751
../../data/raw/wfabba/GOES-16-2021/2021_08_27_239
80042
../../data/raw/wfabba/GOES-16-2021/2021_08_28_240
80099
../../data/raw/wfabba/GOES-16-2021/2021_08_29_241
80570
../../data/raw/wfabba/GOES-16-2021/2021_08_30_242
80847
../../data/raw/wfabba/GOES-16-2021/2021_08_31_243
81036
../../data/raw/wfabba/GOES-16-2021/2021_09_01_244
81718
../../data/raw/wfabba/GOES-16-2021/2021_09_02_245
81771
../../data/raw/wfabba/GOES-16-2021/2021_09_03_246
81799
../../data/raw/wfabba/GOES-16-2021/2021_09_04_247
81817
../../data/raw/wfabba/GOES-16-2021/2021_09_05_248
81838
../../data/raw/wfabba/GOES-16-2021/2021_09_06_249
81913
../../data/raw/wfabba/GOES-16-2021/2021_09_07_250
81999
../../data/raw/wfabba/GOES-16-2021/2021_09_08_251
82048
../../data/raw/wfabba/GOES-16-2021/2021_09_09_252
82124
../../data/raw/wfabba/GOES-16-2021/2021_09_10_25

../../data/raw/wfabba/GOES-16-2021/2022_02_09_040
94850
../../data/raw/wfabba/GOES-16-2021/2022_02_09_040/.ipynb_checkpoints
94913
2925.5202343463898
GOES-17-2021
../../data/raw/wfabba/GOES-17-2021
0
../../data/raw/wfabba/GOES-17-2021/2021_01_20_020
0
../../data/raw/wfabba/GOES-17-2021/2021_01_21_021
10
../../data/raw/wfabba/GOES-17-2021/2021_01_22_022
18
../../data/raw/wfabba/GOES-17-2021/2021_01_23_023
182
../../data/raw/wfabba/GOES-17-2021/2021_01_24_024
194
../../data/raw/wfabba/GOES-17-2021/2021_01_25_025
247
../../data/raw/wfabba/GOES-17-2021/2021_01_26_026
343
../../data/raw/wfabba/GOES-17-2021/2021_01_27_027
400
../../data/raw/wfabba/GOES-17-2021/2021_01_28_028
467
../../data/raw/wfabba/GOES-17-2021/2021_01_29_029
500
../../data/raw/wfabba/GOES-17-2021/2021_01_30_030
527
../../data/raw/wfabba/GOES-17-2021/2021_01_31_031
551
../../data/raw/wfabba/GOES-17-2021/2021_02_01_032
41808
../../data/raw/wfabba/GOES-17-2021/2021_02_02_033
41812
../../data/raw/wfabba/GOES-17-2021/2021_02_0

../../data/raw/wfabba/GOES-17-2021/2021_06_19_170
104793
../../data/raw/wfabba/GOES-17-2021/2021_06_20_171
114180
../../data/raw/wfabba/GOES-17-2021/2021_06_21_172
115457
../../data/raw/wfabba/GOES-17-2021/2021_06_22_173
116149
../../data/raw/wfabba/GOES-17-2021/2021_06_23_174
116889
../../data/raw/wfabba/GOES-17-2021/2021_06_24_175
117761
../../data/raw/wfabba/GOES-17-2021/2021_06_25_176
118433
../../data/raw/wfabba/GOES-17-2021/2021_06_26_177
119092
../../data/raw/wfabba/GOES-17-2021/2021_06_27_178
120643
../../data/raw/wfabba/GOES-17-2021/2021_06_28_179
127724
../../data/raw/wfabba/GOES-17-2021/2021_06_29_180
128924
../../data/raw/wfabba/GOES-17-2021/2021_06_30_181
129391
../../data/raw/wfabba/GOES-17-2021/2021_07_01_182
132494
../../data/raw/wfabba/GOES-17-2021/2021_07_02_183
133299
../../data/raw/wfabba/GOES-17-2021/2021_07_03_184
134239
../../data/raw/wfabba/GOES-17-2021/2021_07_04_185
143755
../../data/raw/wfabba/GOES-17-2021/2021_07_05_186
145681
../../data/raw/wfabba/GOES-17-2

../../data/raw/wfabba/GOES-17-2021/2021_11_09_313
372846
../../data/raw/wfabba/GOES-17-2021/2021_11_10_314
372878
../../data/raw/wfabba/GOES-17-2021/2021_11_11_315
372910
../../data/raw/wfabba/GOES-17-2021/2021_11_12_316
372925
../../data/raw/wfabba/GOES-17-2021/2021_11_13_317
372984
../../data/raw/wfabba/GOES-17-2021/2021_11_14_318
373008
../../data/raw/wfabba/GOES-17-2021/2021_11_15_319
373009
../../data/raw/wfabba/GOES-17-2021/2021_11_16_320
373059
../../data/raw/wfabba/GOES-17-2021/2021_11_17_321
373097
../../data/raw/wfabba/GOES-17-2021/2021_11_18_322
373099
../../data/raw/wfabba/GOES-17-2021/2021_11_19_323
373236
../../data/raw/wfabba/GOES-17-2021/2021_11_20_324
373300
../../data/raw/wfabba/GOES-17-2021/2021_11_21_325
373306
../../data/raw/wfabba/GOES-17-2021/2021_11_22_326
373666
../../data/raw/wfabba/GOES-17-2021/2021_11_23_327
373819
../../data/raw/wfabba/GOES-17-2021/2021_11_24_328
373903
../../data/raw/wfabba/GOES-17-2021/2021_11_25_329
373906
../../data/raw/wfabba/GOES-17-2

../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_18_018
1833
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_19_019
1861
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_20_020
2841
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_21_021
2966
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_22_022
2974
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_23_023
3110
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_24_024
3353
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_25_025
3392
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_26_026
3470
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_27_027
3517
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_28_028
3571
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_29_029
3589
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_30_030
3609
../../data/raw/wfabba/GOES-17-Jan-2021/2021_01_31_031
3625
3160.0011484622955
